# This assignment is to explore and cluster the neighborhoods in Toronto

## Step One:  
Scrape wikipedia page for the table of Neighborhoods
source page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [148]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import lxml
import requests
from IPython.display import display
#pd.options.display.max_columns = None
pd.set_option('display.max_columns', 15)
print('Modules Ready')

Modules Ready


In [71]:
wiki = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wiki, 'lxml')
print(soup.title.text)
print(type(soup))

List of postal codes of Canada: M - Wikipedia
<class 'bs4.BeautifulSoup'>


In [75]:
# Have a quick view of our HTML page

print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":890001695,"wgRevisionId":890001695,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

## Step Two: 
Find the pertinent data from the webpage, and put into a dataframe

In [76]:
# Use soup find the table body
table_body = soup.find('tbody')
#print(table_body)

In [131]:

# found help for this at https://datascience.stackexchange.com/questions/10857/how-to-scrape-a-table-from-a-webpage
print(len(rows))
print(type(rows))
#print(rows)

289
<class 'bs4.element.ResultSet'>


In [163]:
# Create empty dataframe to store the data 
rows = table_body.find_all('tr')
df_cols = ['Postcode', 'Borough', 'Neighbourhood']
pc = pd.DataFrame(columns = df_cols)
print('starting with', len(rows), 'rows \n')


# iterate through rows, appending into dataframe
for row in rows:
    cols = row.find_all('td')
    cols = [x.text.strip() for x in cols]
    try:
        # Need a test to catch where no value exists (such as case with first line )
        if cols[0]:
            pc = pc.append({'Postcode':cols[0],
                    'Borough':cols[1],
                    'Neighbourhood':cols[2]}, ignore_index=True)
    except:
        print('hit an empty row, moving on \n')

print('output is a dataframe of this shape: ', pc.shape)
print('first dataframe\n\n', pc)

starting with 289 rows 

hit an empty row, moving on 

output is a dataframe of this shape:  (288, 3)
first dataframe

     Postcode           Borough  \
0        M1A      Not assigned   
1        M2A      Not assigned   
2        M3A        North York   
3        M4A        North York   
4        M5A  Downtown Toronto   
5        M5A  Downtown Toronto   
6        M6A        North York   
7        M6A        North York   
8        M7A      Queen's Park   
9        M8A      Not assigned   
10       M9A         Etobicoke   
11       M1B       Scarborough   
12       M1B       Scarborough   
13       M2B      Not assigned   
14       M3B        North York   
15       M4B         East York   
16       M4B         East York   
17       M5B  Downtown Toronto   
18       M5B  Downtown Toronto   
19       M6B        North York   
20       M7B      Not assigned   
21       M8B      Not assigned   
22       M9B         Etobicoke   
23       M9B         Etobicoke   
24       M9B         Etobicoke

## Step Three: 
Clean up the dataframe

In [204]:
# First remove rows where Borough is 'Not assigned'
cleanpc = pc[pc.Borough != 'Not assigned']
cleanpc.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [206]:
# Next, If a cell has a borough but a Not assigned neighborhood, set neighborhood to same as the borough
cleanpc['Neighbourhood'] = np.where(cleanpc['Neighbourhood'] == 'Not assigned', cleanpc['Borough'], cleanpc['Neighbourhood'])
cleanpc

/anaconda3/anaconda2/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [207]:
# Now combine Neighbourhood field when PostCode is the same
grouppc = cleanpc.groupby(['Postcode'])['Neighbourhood'].apply(','.join).reset_index()

## Step Four: 
Show the final shape of the dataframe.  Also see the first few rows:

In [210]:
grouppc.head()

,Postcode,Neighbourhood
0,M1B,"Rouge,Malvern"
1,M1C,"Highland Creek,Rouge Hill,Port Union"
2,M1E,"Guildwood,Morningside,West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


In [211]:
grouppc.shape

(103, 2)